# MFT Asynchronous QC

In [1]:
_period = "LHC22o"
_type = "data"
_pass1 = "apass1"
_pass2 = "apass2"
_run = 526606

In [2]:
import ROOT
from ROOT import gStyle
%jsroot on

print(f"period: {_period}")
print(f"type:   {_type}")
print(f"pass1:  {_pass1}")
print(f"pass2:  {_pass2}")
print(f"run:    {_run}")

# global variables:
_f1 = ROOT.TFile.Open(f"Results/{_period}/runsRootFiles/{_run}_{_pass1}.root")
_f2 = ROOT.TFile.Open(f"Results/{_period}/runsRootFiles/{_run}_{_pass2}.root")

_names = [
# track position (6 in total)
"mMFTTrackEtaPhi_5_MinClusters",
"mMFTTrackXY_5_MinClusters",
"mMFTTrackEtaPhi_7_MinClusters",
"mMFTTrackXY_7_MinClusters",
"mMFTTrackEtaPhi_8_MinClusters",
"mMFTTrackXY_8_MinClusters",
# tracks (11 in total)
"mMFTTrackEta",
"mMFTTrackNumberOfClusters",
"mMFTTrackPhi",
"mMFTTrackTanl",
"mNOfTracksTime",
"mMFTTrackInvQPt",
"mMFTCATrackPt",
"mMFTLTFTrackPt",
"mMFTCATrackEta",
"mMFTLTFTrackEta",
"mMFTTracksBC",
# clusters (4 in total)
"mMFTClusterPatternIndex",
"mMFTClusterSensorIndex",
"mMFTClustersROFSize",
"mNOfClustersTime"]

# which histograms to plot with log scale on the y-axis
_logScales = [
"mMFTCATrackPt",
"mMFTLTFTrackPt",
"mMFTClusterPatternIndex",
"mMFTClusterSensorIndex",
"mMFTClustersROFSize"]

gStyle.SetTitleX(0.3)
gStyle.SetTitleY(1.0)
gStyle.SetOptStat(0)

def createArrOfLegends(length):
    arr = ROOT.TObjArray(length)
    arr.SetOwner()
    for i in range(length):
        l = ROOT.TLegend(0.60,0.90,0.95,1.00)
        l.SetBorderSize(0)
        l.SetFillColor(0)
        l.SetFillStyle(0)
        l.SetTextFont(42)
        arr.AddAt(l,i)
    return arr

arrLeg1D = createArrOfLegends(15)
arrLeg2D = createArrOfLegends(18)
#arrLegRatios = createArrOfLegends(21) 

def createArrOfRatioHistos(idxFrom, idxToPlusOne):
    arr = ROOT.TObjArray(idxToPlusOne-idxFrom)
    arr.SetOwner()
    for i in range(idxFrom,idxToPlusOne):
        h1 = _f1.Get(_names[i])
        h2 = _f2.Get(_names[i])
        hRatio = h2.Clone(f"ratio_{_names[i]}")
        hRatio.Divide(h1)
        arr.AddAt(hRatio,i)
    return arr

#arrHistRatios = createArrOfRatioHistos(0,21)

def plotComparison1D(canvasName, drawOption, nCols, nRows, histoNames, legFirstIdx):
    c = ROOT.TCanvas(canvasName,"",nCols*400,nRows*300)
    c.Divide(nCols,nRows)
    i = 1
    for histoName in histoNames:
        h1 = _f1.Get(histoName)
        h2 = _f2.Get(histoName)
        norm1 = 0
        norm2 = 0
        max1 = 0
        max2 = 0
        if h1 != None:
            norm1 = h1.Integral()
            if norm1 > 0:
                h1.Scale(1./norm1)
            max1 = h1.GetMaximum()
        if h2 != None:
            norm2 = h2.Integral()
            if norm2 > 0:
                h2.Scale(1./norm2)
            max2 = h2.GetMaximum()
        maximum = max1
        if max2 > max1:
            maximum = max2
        c.cd(i)
        logScale = False
        for name in _logScales:
            if histoName == name:
                logScale = True
        if logScale == True:
            c.cd(i).SetLogy()
        l = arrLeg1D.At(legFirstIdx)
        if h1 != None:
            l.AddEntry(h1,f"{_pass1}: {norm1:.3e}")
            h1.SetLineColor(ROOT.kBlue)
            if logScale == False:
                h1.GetYaxis().SetRangeUser(0.,maximum*1.05)
            h1.Draw(f"{drawOption}")
        if h2 != None: 
            l.AddEntry(h2,f"{_pass2}: {norm2:.3e}")
            h2.SetLineColor(ROOT.kRed)
            h2.Draw(f"{drawOption} same")
        l.Draw()
        i+=1
        legFirstIdx+=1
    return c

Welcome to JupyROOT 6.26/10
period: LHC22o
run:    526606
pass1:  apass1
pass2:  apass2


## TRACK POSITION:

In [3]:
nCols = 2
nRows = 6
c = ROOT.TCanvas("trackPos","",nCols*400,nRows*300)
c.Divide(nCols,nRows)
empty = ROOT.TObject()

i = 1
for histoName in _names[0:6]:
    h1 = _f1.Get(histoName)
    h2 = _f2.Get(histoName)
    #c.cd(3*i-2).SetLogz()
    c.cd(2*i-1)
    if h1 != None:
        h1.Draw("colz")
    l1 = arrLeg2D.At(3*i-3)
    l1.AddEntry(empty,f"{_pass1}: {h1.Integral():.3e}","")
    l1.Draw()
    #c.cd(3*i-1).SetLogz()
    c.cd(2*i)
    if h2 != None:
        h2.Draw("colz")
    l2 = arrLeg2D.At(3*i-2)
    l2.AddEntry(empty,f"{_pass2}: {h2.Integral():.3e}","")
    l2.Draw()
    #c.cd(3*i).SetLogz()
    #c.cd(3*i)
    #hRat = arrHistRatios.At(i-1)
    #hRat.Draw("colz")
    #lRat = arrLeg2D.At(3*i-1)
    #lRat.AddEntry(empty,f"ratio {_pass2}/{_pass1}","")
    #lRat.Draw()
    i+=1
c.Draw()

## TRACK PROPERTIES:

In [4]:
c2 = plotComparison1D("trackProp","hist",2,6,_names[6:17],0)
c2.Draw()

## CLUSTER PROPERTIES:

In [5]:
c3 = plotComparison1D("clusterProp","hist",2,2,_names[17:21],11)
c3.Draw()